Kareef Ullah-rising senior at Wootton High School
<br>
Worked in the lab from June 17th-August 22nd
<br>
Focused on creating simulations/other miscellaneous work
<br>
3 important simulations

First couple of weeks: 
<br>
Introduced to git/github/python/VScode and graspy package (go through tutorial on https://graspy.neurodata.io/tutorial)
<br>
Learned process of git add -> git commit -> git push
<br>
Made small PR to graspy to make tutorial for Adjacency Spectral Embed functional
<br>
Read papers: Connectal coding, CASC, Two Truths

# LDT simulation
<br>
Purpose: To demonstrate issues with the latent distribution (nonpar) test when the 2 graphs have different number of vertices
<br>
Shows type 1 error from the nonpar test on 2 block graphs with varying number of vertices
<br>
Concluded that type 1 error is more common when:
- There is a large difference between the number of vertices of the two graphs 
<br>
- The number of vertices for either of the graphs is low
<br>
Code for LDT simulation below

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas import DataFrame

from graspy.inference import LatentDistributionTest
from graspy.simulations import sbm
from graspy.utils import symmetrize
%matplotlib inline

In [ ]:
np.random.seed(8888)

B = [[0.5, 0.2], [0, 0.05]]
B = symmetrize(B)
k = 2
tests = 10
start = 50
stop = 200
diff1 = 50
diff2 = 100
reps = 10
alpha = 0.05
ns = []
ms = []
newms = []
error_list = []
temp = []

In [ ]:
for n in range(start, stop, diff1):
    ns.append(n)
    for m in range(n, n + (diff2 * reps), diff2):
        print(f"Running tests for n={n}, m={m}")
        cn = [n // k] * k
        cm = [m // k] * k
        A1 = sbm(cn, B)
        A2 = sbm(cm, B)
        valid = 0
        ldt = LatentDistributionTest(n_components=2)
        for _ in range(tests):
            p = ldt.fit(A1, A2)
            if p < 0.05:
                valid += 1
        error = valid / tests
        print(f"Error was {error}")
        temp.append(error)
        ms.append(m - n)
    error_list.append(temp)
    temp = []

for num in ms:
    if num not in newms:
        newms.append(num)

In [ ]:
df = DataFrame(error_list, index=ns, columns=newms)
sns.heatmap(df, annot=True, linewidths=0.5)
plt.title("Variation of Type 1 Error with Different Cases of LDT")
plt.xlabel("m - n")
plt.ylabel("n")

# Do's and don't experiment 7 simulations
<br>
Two populations, sample DCSBMs from them
<br>
For pop1, the promiscuity parameter is 0.5 for all verts
<br>
For pop2, vertex 1 has a different promiscuity parameter
<br>
Use omnibus embedding and DCorr test to see whether each vertex is different
<br>
Purpose: Want to make sure that DCorr test yields p-values consistent to prediction (significant p-value for vertex 1 but insignificant for all others)
<br>
Simulations to see distribution of p-values for a single node and save p-values for all nodes
<br>
Code for experiment 7 simulations below (node-wise)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import Parallel, delayed
from mgcpy.hypothesis_tests.transforms import k_sample_transform
from mgcpy.independence_tests.dcorr import DCorr
from tqdm import tqdm
from graspy.embed import OmnibusEmbed
from graspy.plot import heatmap, pairplot
from graspy.simulations import p_from_latent, sample_edges
from graspy.utils import cartprod
from src.utils import n_to_labels

In [ ]:
def block_to_full(block_mat, inverse, shape):
    block_map = cartprod(inverse, inverse).T
    mat_by_edge = block_mat[block_map[0], block_map[1]]
    full_mat = mat_by_edge.reshape(shape)
    return full_mat


def dcsbm(vertex_assignments, block_p, degree_corrections, return_p_mat=False):
    n_verts = len(vertex_assignments)
    p_mat = block_to_full(block_p, vertex_assignments, (n_verts, n_verts))
    p_mat = p_mat * np.outer(degree_corrections, degree_corrections)
    if return_p_mat:
        return p_mat
    else:
        return sample_edges(p_mat, directed=False, loops=True)


def sample_graph(latent):
    p = p_from_latent(latent, rescale=False, loops=False)
    return sample_edges(p, directed=False, loops=False)


def compute_t_stat(sample1, sample2):
    test = DCorr()
    u, v = k_sample_transform(sample1, sample2, is_y_categorical=False)
    return test.test_statistic(u, v)[0]


def node_wise_2_sample(latent, node_ind):
    node_latent_pop1 = np.squeeze(latent[:n_graphs, node_ind, :])
    node_latent_pop2 = np.squeeze(latent[n_graphs:, node_ind, :])
    t_stat = compute_t_stat(node_latent_pop1, node_latent_pop2)
    return t_stat


def compute_pop_t_stats(pop_latent):
    n_verts = pop_latent.shape[1]
    t_stats = np.zeros(n_verts)
    for node_ind in range(n_verts):
        t_stat = node_wise_2_sample(pop_latent, node_ind)
        t_stats[node_ind] = t_stat
    return t_stats


def bootstrap_population(latent, n_graphs, seed):
    np.random.seed(seed)
    bootstrapped_graphs = []
    for g in range(n_graphs):
        graph = sample_graph(latent)
        bootstrapped_graphs.append(graph)

    omni = OmnibusEmbed(n_components=2)
    bootstrapped_latent = omni.fit_transform(bootstrapped_graphs)
    bootstrap_t_stats = compute_pop_t_stats(bootstrapped_latent)
    return bootstrap_t_stats

In [ ]:
block_p = np.array([[0.25, 0.05], [0.05, 0.15]])
n_graphs = 4
diff = 1
verts_per_block = 64
n_verts = 2 * verts_per_block
n = 2 * [verts_per_block]
node_labels = n_to_labels(n).astype(int)
temp = []
node1 = []
node_list = []

# test settings
sims = 20
n_bootstraps = 50
n_jobs = -2
verbose = True

In [ ]:
for x in tqdm(range(sims)):
    if verbose:
        print(f"Running simulation {x+1}")

    if verbose:
        print("Generating graph populations")
    vertex_assignments = np.zeros(n_verts, dtype=int)
    vertex_assignments[verts_per_block:] = 1
    degree_corrections = np.ones(n_verts)

    # Population 1
    graphs_pop1 = []
    for i in range(n_graphs):
        graphs_pop1.append(dcsbm(vertex_assignments, block_p, degree_corrections))
    graphs_pop1 = np.array(graphs_pop1)

    # Population 2
    degree_corrections[0] += diff
    degree_corrections[1:verts_per_block] -= diff / (verts_per_block - 1)

    graphs_pop2 = []
    for i in range(n_graphs):
        graphs_pop2.append(dcsbm(node_labels, block_p, degree_corrections))
    graphs_pop2 = np.array(graphs_pop2)

    n_components = 2

    if verbose:
        print("Doing Omnibus Embedding")
    omni = OmnibusEmbed(n_components=n_components, algorithm="randomized")
    graphs = np.concatenate((graphs_pop1, graphs_pop2), axis=0)
    pop_latent = omni.fit_transform(graphs)
    print(pop_latent.shape)

    # Bootstrapping
    if verbose:
        print(f"Running {n_bootstraps} bootstraps")

    avg_latent = np.mean(pop_latent, axis=0)

    def bsp(seed):
        return bootstrap_population(avg_latent, n_graphs * 2, seed)

    seeds = np.random.randint(1e8, size=n_bootstraps)
    out = Parallel(n_jobs=n_jobs, verbose=verbose)(delayed(bsp)(seed) for seed in seeds)
    nulls = np.array(out).T
    
    
    sample_t_stats = compute_pop_t_stats(pop_latent)
    node_p_vals = np.zeros(n_verts)
    for i, sample_t in enumerate(sample_t_stats):
        num_greater = len(np.where(sample_t < nulls[i, :])[0])
        p_val = num_greater / n_bootstraps
        if p_val < 1 / n_bootstraps:
            p_val = 1 / n_bootstraps
        node_p_vals[i] = p_val
    node1.append(node_p_vals[1])
    node_list.append(node_p_vals)
#save all nodes
print(node_list)

#display distribution of p-vals for a single node
node1=pd.Series(node1, name="p_vals")
sns.distplot(node1)

# Comparison of graph embedding/clustering methods for SBMs
Purpose: Want to see which embedding/clustering methods work better on sbms with different settings of # of blocks and B matrix 
<br>
Go through list of n_verts
<br>
B matrix of k blocks with diagonal values p and other values q
<br>
Have k constant or increase linearly with n_verts, have q constant or decrease with n_verts
<br>
Create sbm and embed with either ASE, LSE, or LGC (local graph clustering)
<br>
Cluster and compute average ARI and standard error of ARI over a number of sims with KMeans or GMM
<br> 
Concluded that k does not affect ARI as much as q
<br>
Code for diff embed simulation below (LGC not included)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
from scipy import stats
from graspy.simulations import sbm
from graspy.embed import AdjacencySpectralEmbed, LaplacianSpectralEmbed
from graspy.plot import heatmap
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from src.utils import n_to_labels

In [ ]:
#2 methods for k
constant_k = 2
def linear_k(slope, n):
    k = n / slope
    k = int(k)
    return k

#2 methods for q
constant_q = 0.5
def decay_q(slope, n):
    q = slope / n
    return q

#Generate B Matrix
def B_matrix(k, p, q):
    B = np.zeros((k,k))
    np.fill_diagonal(B, p) 
    B[B == 0] = q
    return B

def avg_ari(slope, n_verts, n_sims, p, embed, const_k=True, const_q=True):
    temp = []
    ari_vals = []
    stand_error = []
        
    #Generate graph
    for n in n_verts:
        for _ in range(n_sims):
            
            if const_k:
                k, k_func = constant_k
            else:
                k = linear_k(slope, n)
                k_func = f"n_verts / {slope}"
                
            if const_q:
                q, q_func = constant_q
            else:
                q = decay_q(slope, n)
                q_func = f"{slope} / n_verts"
                
            B = B_matrix(k, p, q)
            cn = [n//k] * k
            labels_sbm = n_to_labels(cn).astype(int)
            G = sbm(cn, B)

            #embedding and clustering
            Xhat = embed.fit_transform(G)
            
            #can be KMeans or GMM
            clust = KMeans(n_clusters = k)
            labels_clust = clust.fit_predict(Xhat)
            ari = adjusted_rand_score(labels_sbm, labels_clust)
            temp.append(ari)
            print("n_verts: {} ARI: {}".format(n, ari))

        ari_vals.append(np.sum(temp) / n_sims)
        stand_error.append(stats.sem(temp))
        temp = []     
    return ari_vals, stand_error, k_func, q_func

In [ ]:
n_verts = [80, 120, 160, 200, 240, 280, 320, 360, 400]
slope = n_verts[0] / 2
p = 0.5
n_sims = 30

In [ ]:
embed_ase = AdjacencySpectralEmbed()
embed_lse = LaplacianSpectralEmbed()

ari_vals_ase, stand_error_ase, k_func, q_func = avg_ari(slope, n_verts, n_sims, p, embed_ase, const_k=False, const_q=False)
ari_vals_lse, stand_error_lse, _, _ = avg_ari(slope, n_verts, n_sims, p, embed_lse, const_k=False, const_q=False)

plt.errorbar(n_verts, 
             ari_vals_ase, 
             yerr=stand_error_ase,
             marker='s',
             mfc='red',
             label="ASE")
plt.errorbar(n_verts, 
             ari_vals_lse, 
             yerr=stand_error_lse,
             marker='s',
             mfc='blue',
             label="LSE")
plt.title(f"k = {k_func}, p = {p}, q = {q_func}") 
plt.xlabel("n_verts")
plt.xticks(n_verts)
plt.ylabel("ARI")
plt.legend(loc='upper left')